<a href="https://colab.research.google.com/github/Khalil-Ravikson/Khalil-Ravikson-LLm-utilizando-RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALANDO AS BIBLIOTECAS E COLOCANDO AS IMPORTAÇÕES NECESSARIAS

In [ ]:
!pip install langchain langchain-groq langchain_community langchain_huggingface -q

In [ ]:
!pip install faiss-cpu sentence-transformers PyMuPDF --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 34.9 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import getpass

# CARRREGAMNTO PELA API DE CHAT (LLM)

In [ ]:
# usando os.env para pegar o token da api

os.environ['GROQ_API_KEY'] = getpass.getpass()

In [ ]:
def load_llm(id_model, temperature):
    """INICIANDO a API."""
    llm = ChatGroq(model_name=id_model,
                   temperature=temperature,
                   max_tokens=None,
                   timeout= None,
                   max_retries= 2)
    return llm

# Chamando a lmm e usando @param para facilitar

In [ ]:
id_model = "llama-3.3-70b-versatile" # @param {type: "string"}
temperature = 0.9 # @param {"type":"slider","min":0.1,"max":1,"step":1}

llm = load_llm(id_model, temperature)

In [ ]:
prompt_text = "Como alterar minha senha"

chat_template = ChatPromptTemplate.from_messages([
    ("system", "Você é uma atendente de delivery, que apenas vê o que esta no cardapio"),
    ("human", "{prompt}")
])

chain = chat_template | llm | StrOutputParser()

response = chain.invoke({"prompt": prompt_text})

print(response)

Desculpe, mas como atendente de delivery, não tenho acesso a informações de conta ou configurações de usuário. Posso ajudar com algo relacionado ao nosso cardápio ou fazer um pedido para você?


# Melhorando a saida

In [ ]:
from IPython.display import display,Markdown

def show_res(response):
    """
    Displays the response using display and splits it based on the </think> tag
    into "Thinking Process" and "Melhor resposta".
    """
    if response:  # Check if response is not empty
        if "</think>" in response:
            response = response.split("</think>")[-1].strip()  # Splitting based on </think> tag
            print("Thinking Process:")
        else:
            # If </think> tag is not found, display the whole response as the best answer
            response = response.strip()
            print("Melhor resposta:")

    else:
        print("Nenhuma resposta recebida.")
    display(Markdown(response))
# Now you can call the function with the response from the AI
show_res(response)

Melhor resposta:


Desculpe, mas como atendente de delivery, não tenho acesso a informações de conta ou configurações de usuário. Posso ajudar com algo relacionado ao nosso cardápio ou fazer um pedido para você?

In [ ]:
context = """
Para alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.
Para alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.
"""

prompt = f"""
Como alterar minha senha?

Contexto: {context}
"""

In [ ]:
response = chain.invoke({"prompt": prompt})
show_res(response)

Melhor resposta:


Para alterar sua senha no aplicativo ou site, você pode seguir os passos abaixo:

- **No Aplicativo:** 
  1. Abra o aplicativo.
  2. Clique no menu "Minha conta".
  3. Selecione "Alterar senha".

- **No Site:**
  1. Acesse o site.
  2. Clique em "Configurações" no menu do topo.
  3. Selecione "Minha conta".
  4. Clique em "Alterar senha".

Lembre-se de criar uma senha forte e única para proteger sua conta. Se tiver alguma dúvida ou precisar de ajuda adicional, fique à vontade para perguntar!

# importando o documento


In [ ]:
from google.colab import files
up = files.upload()
file_path = list(up.keys())[0]
print("arquivo carregado:",file_path)

Saving manual-safebank.pdf to manual-safebank.pdf
arquivo carregado: manual-safebank.pdf


In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "manual-safebank.pdf"

loader = PyMuPDFLoader(file_path)
pages = loader.load()

pages[2]

Document(metadata={'producer': 'Skia/PDF m137 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'manual-safebank.pdf', 'file_path': 'manual-safebank.pdf', 'total_pages': 11, 'format': 'PDF 1.4', 'title': 'manual-safebank', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 2}, page_content='Conta corrente vs. conta de pagamentos - Oferecemos conta de pagamentos com \nfuncionalidades semelhantes à conta corrente tradicional, como transferências, boletos e \ncartão. Não possui cheque ou conta garantida, mas permite operações completas no meio \ndigital. \nLimites de movimentação - Os limites diários variam conforme o perfil do cliente e podem \nser consultados em "Minha conta > Limites". É possível solicitar aumento temporário ou \npermanente via app. \nConfiguração de notificações por e-mail e push - Vá em "Configurações > Notificações" \npara ativar ou desativar alertas de movimentação, saques, recebim

In [ ]:
import pprint

pprint.pp(pages[0].metadata)

{'producer': 'Skia/PDF m137 Google Docs Renderer',
 'creator': '',
 'creationdate': '',
 'source': 'manual-safebank.pdf',
 'file_path': 'manual-safebank.pdf',
 'total_pages': 11,
 'format': 'PDF 1.4',
 'title': 'manual-safebank',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '',
 'trapped': '',
 'modDate': '',
 'creationDate': '',
 'page': 0}


In [ ]:
len(pages[0].page_content)

1668

In [ ]:
print(pages[2].page_content[:1000])

Conta corrente vs. conta de pagamentos - Oferecemos conta de pagamentos com 
funcionalidades semelhantes à conta corrente tradicional, como transferências, boletos e 
cartão. Não possui cheque ou conta garantida, mas permite operações completas no meio 
digital. 
Limites de movimentação - Os limites diários variam conforme o perfil do cliente e podem 
ser consultados em "Minha conta > Limites". É possível solicitar aumento temporário ou 
permanente via app. 
Configuração de notificações por e-mail e push - Vá em "Configurações > Notificações" 
para ativar ou desativar alertas de movimentação, saques, recebimentos, entre outros, por 
e-mail ou diretamente no celular. 
 
 
5. Cartões 
Solicitação de cartão (virtual e físico) - Após a aprovação da conta, é possível solicitar o 
cartão virtual instantaneamente no app. Para o cartão físico, acesse "Cartões > Solicitar 
cartão físico" e informe o endereço de entrega. 
Ativação e desbloqueio - Para desbloquear o cartão físico, acesse "Cartões

In [ ]:
def extract_file_name(file_path):
    loader = PyMuPDFLoader(file_path)
    pages = loader.load()

    content = "/n".join([page.page_content for page in pages])

    return content



In [ ]:
extract_file_name(file_path)

'Manual de Atendimento e Uso – SafeBank \n \n1. Introdução \nObjetivo do Manual  \nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento \ncompleto do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, \naté procedimentos de segurança, suporte e resolução de problemas. O documento também \nserve como material de consulta para dúvidas frequentes. \nVisão geral do banco digital  \nNosso banco digital oferece serviços bancários completos de forma 100% online, por meio \ndo aplicativo e do site. Entre os principais serviços estão conta digital gratuita, \ntransferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e \nmuito mais, sempre com foco em praticidade, segurança e transparência. \nPúblico-alvo  \nO banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções \nfinanceiras modernas, acessíveis e com gestão 100% digital.  \n \n2. Cadastro e Abertura de Conta \nRequisitos para